# Working with tabular data in Python

One of the foundational parts of any astronomical data analysis workflow is the input, output, and manipulation of tables of data.  This can range from collating the results of other analysis to downloading/exploring large catalogs of data to formatting LaTeX tables in manuscripts for publication.  The Python ecosystem provides a number of powerful tools for doing this.  The most popular of these is Pandas (http://pandas.pydata.org) which has become very widely used in science in general.  The ``astropy.io.*`` modules provide a set of utilities for handling a wide variety of ASCII (e.g. CSV, ECSV) and binary (e.g. FITS, HDF5) data types. AstroPy's [Table](http://docs.astropy.org/en/stable/table/index.html) provides support for manipulating heterogeneous data tables and integrates with AstroPy's [Units and Quantities](http://astropy.readthedocs.org/en/stable/units/index.html) functionality. 

A more in-depth look at AstroPy Tables and what it can do will be given on Thursday by Tom Aldcroft. This tutorial will focus more on Pandas and how it can interoperate with AstroPy functionality.

In [ ]:
# generic imports
import numpy as np
import matplotlib
from matplotlib import style
style.use('ggplot')  
matplotlib.use('nbagg')  # required for interactive plotting
import matplotlib.pyplot as plt

## Getting started with Pandas

A great introduction to Pandas functionality is the [10 Minutes to Pandas](http://pandas.pydata.org/pandas-docs/stable/10min.html) tutorial. A complementary Jupyter notebook that covers Pandas basics in detail can be found [here](https://github.com/donnemartin/data-science-ipython-notebooks/blob/master/pandas/pandas.ipynb). 

There are also a number of astronomy-oriented tutorials that feature Pandas:

* Jane Rigby is setting up a one-stop shop for such tutorials/examples at https://github.com/janerigby/astro-pandas-tutorials.
* Michael Gully-Santiago has some examples of using Pandas to explore Gaia data at https://github.com/gully/adrasteia.
* A trove of brown dwarf research-related Jupyter notebooks, many of which featuring Pandas, can be found at https://github.com/BrownDwarf/ApJdataFrames. 

This tutorial borrows heavily from these resources and will emphasize how to quickly get from input data to real scientific analysis using Pandas and AstroPy.

In [ ]:
import pandas as pd  # this is the accepted convention for importing pandas, much as np is the convention for numpy

Reading in basic ASCII data is straightforward using the ``read_table()`` and ``read_csv()`` methods:

In [ ]:
# read in some background data courtest of Jane Rigby
bkgd = pd.read_table("data/zody_and_ism.txt", comment='#', delim_whitespace=True)
bkgd.head()

In [ ]:
bkgd['ISM'].head()

For quick-n-dirty data exploration, the ability to plot directly from DataFrame objects is very handy. Especially when coupled with the ``nbagg`` interactive backend which allows you to pan and zoom:

In [ ]:
bkgd.plot(x='wave')
plt.show()

In [ ]:
bkgd.plot?

In [ ]:
bkgd.plot(x='wave', y='ISM', logy=True)
plt.show()

We can read another background component from another file and incorporate it into our DataFrame using ``merge()``:

In [ ]:
thermal_bg = pd.read_csv("data/thermal_curve_jwst_jrigby_1.1.csv", comment="#", names=("wave", "thermal"))
thermal_bg.head()

In [ ]:
bg_merge = pd.merge(bkgd, thermal_bg, on='wave', how='outer')  # keep all data...
bg_merge.plot(x='wave', logy=True)
plt.show()

In [ ]:
bg_merge.head()

In [ ]:
bg_merge[275:280]

In [ ]:
bg_merge.info()

### Exercise

Make a new column that is the sum of the background components and plot the results. (HINT: It's really easy! Access each column by name like with a dict...)

In [ ]:
# %load solution1.py

## More complicated data formats

[``astropy.io.ascii``](http://docs.astropy.org/en/stable/io/ascii/index.html) provides support for a wide variety of ASCII formats, including many astronomy-specific ones. These include:

* _Basic_: basic table with customizable delimiters and header configurations
* _Cds_: CDS format table (also Vizier and ApJ machine readable tables)
* _Daophot_: table from the IRAF DAOphot package
* _Ecsv_: Enhanced CSV format
* _FixedWidth_: table with fixed-width columns (see also Fixed-width Gallery)
* _Ipac_: IPAC format table
* _HTML_: HTML format table contained in a ``<table>`` tag
* _Latex_: LaTeX table with datavalue in the tabular environment
* _Rdb_: tab-separated values with an extra line after the column definition line
* _SExtractor_: SExtractor format table

The reader is also fairly smart and can auto-detect many of these formats.  Here's an example using a table from an ApJ paper (courtesty Jane Rigby):

In [ ]:
!head -100 data/apjsaa2f41t6_mrt.txt

In [ ]:
from astropy.io import ascii
table = ascii.read("data/apjsaa2f41t6_mrt.txt")
table.show_in_notebook()

``ascii.read()`` returns an [``astropy.table.Table``](http://docs.astropy.org/en/stable/io/ascii/index.html) instance! We'll come back to cover ``Table`` in a bit more detail. For now we'll take advantage of the ``to_pandas()`` method to convert the table into a pandas DataFrame:

In [ ]:
df = table.to_pandas()
df.head()

In [ ]:
df.mean()

In [ ]:
df.describe()

For data like this, we can reset the index to be the ``ID`` so that we can more easily access data for specific objects:

In [ ]:
df.set_index('ID', inplace=True)

In [ ]:
df.head()

In [ ]:
df.ix['MMT10']

## Exercise

Create a new column that contains the [O III] 4363/5007 flux ratio and plot the flux ratio vs. [O III] 5007 flux. 

(HINT: Look in the header of the data file shown above for descriptions of the columns and use the ``kind='scatter'`` argument in the ``plot()`` command)

In [ ]:
# %load solution2.py

Now we can use this new column to make a mask to define a subset of the data with strong 4363 line emission:

In [ ]:
mask = df['O3-4363'].gt(df['O3-4363'].median())
subset = df[mask].copy()
subset.describe()

Masks can be combined logically to make more sophisticated filtering possible:

In [ ]:
mask2 = df['f_O3-4363'] != 'd'  # exclude where 4363 line is contaminated by sky emission
subset2 = df[mask & mask2].copy()  # combine with the previous mask to make a new subset with cleaner 4363
subset2.describe()

## Binary tables

Pandas has native support for binary tables in HDF5 and even Excel format. However, it does not support the most popular binary format used in astronomy, FITS.  

Again, AstroPy ``Table`` can be used as a bridge:

In [ ]:
from astropy.table import Table

In [ ]:
fits_df = Table.read("data/cdfs_obs_sources.fits").to_pandas()
fits_df.head()

In [ ]:
fits_df.describe()

When loading FITS tables, it's recommended to use the high-level ``Table`` interface rather than the lower level ``astropy.io.fits`` interface unless you really know what you're doing.  The ``Table`` I/O interface automatically handles the endianness of the data and converts accordingly, as needed. 

# Tables vs DataFrames

As we've seen so far, Pandas is a very powerful, flexible tool for analyzing and visualizing tabular data.  Combining it with AstroPy's support for astronomy-specific data formats makes it especially handy for working with astronomical data. 

What DataFrames are not, however, are general purpose tables.  Each column must be of a specific data type and only certain data types are supported.  Multidimensional columns are also not supported, though they are by the complementary package [xarray](http://xarray.pydata.org/en/stable/). AstroPy Tables are much more general purpose and while the ability to convert between Tables and Pandas via the ``to_pandas()`` and ``from_pandas()`` methods works well, there are some caveats:

 - Tables with multi-dimensional columns cannot be converted.
 - Masked values are converted to `numpy.nan`. Numerical columns, int or float, are thus converted to ``numpy.float`` while string columns with missing values are converted to object columns with ``numpy.nan`` values to indicate missing or masked data. Therefore, one cannot always round-trip between `Table` and `DataFrame`.
 
AstroPy Tables also support the use of Units, Quantities, and SkyCoords.  The ``QTable`` object is provided which is exactly like ``Table``, but ``Quantity`` is used for columns that have defined units.

In [ ]:
from astropy.table import QTable
import astropy.units as u
from astropy.time import Time

In [ ]:
t = Table()
t['index'] = [1, 2]
t['time'] = Time(['2001-01-02T12:34:56', '2001-02-03T00:01:02'])
t['velocity'] = [3, 4] * u.m / u.s
t

In [ ]:
type(t['velocity'])

In [ ]:
t['velocity'].unit

In [ ]:
qtab = QTable(t)
type(qtab['velocity'])

In [ ]:
(t['velocity'] ** 2).unit  # wrong

In [ ]:
(qtab['velocity'] ** 2).unit  # right!

In [ ]:
qtab.to_pandas()  # oops, can't do this...